# Задание по программированию: Примеры задач анализа текстов

## Классификация текстов: спам-фильтр для SMS

В этом задании вам предстоит взять открытый датасет с SMS-сообщениями, размеченными на спам ("spam") и не спам ("ham"), построить на нем классификатор текстов на эти два класса, оценить его качество с помощью кросс-валидации, протестировать его работу на отдельных примерах, и посмотреть, что будет происходить с качеством, если менять параметры вашей модели.

In [1]:
from __future__ import division, print_function

import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Задание 1

Загрузите датасет. Описание датасета можно посмотреть здесь: http://www.dt.fee.unicamp.br/~tiago/smsspamcollection

In [2]:
# smsspam collection.zip

### Задание 2

Считайте датасет в Python (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.

In [9]:
with open(file='SMSSpamCollection.txt', mode='r') as file:
    sms_data = file.read().split('\n') 

In [45]:
sms_data[-5:]

['ham\tWill ü b going to esplanade fr home?',
 'ham\tPity, * was in mood for that. So...any other suggestions?',
 "ham\tThe guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free",
 'ham\tRofl. Its true to its name',
 '']

### Задание 3

Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

In [54]:
#split the spam/ham from the lines 
new_text = [line.split('\t') for line in sms_data]

# split the sms text from the new text
sms_text = [new_text[line][1] for line in range(len(new_text)-1)]

# split spam/ham labels from the new text 
sms_label = [new_text[line][0] for line in range(len(new_text)-1)]

# change spam/ham lebels into the 0/1 format
sms_label_bi = [0 if t=='ham' else 1 for t in sms_label]

In [56]:
sms_label[:5]
sms_label_bi[:5]
sms_text[:5]

['ham', 'ham', 'spam', 'ham', 'ham']

[0, 0, 1, 0, 0]

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
 'Ok lar... Joking wif u oni...',
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
 'U dun say so early hor... U c already then say...',
 "Nah I don't think he goes to usf, he lives around here though"]

In [57]:
sms_label[-5:]
sms_label_bi[-5:]
sms_text[-5:]

['spam', 'ham', 'ham', 'ham', 'ham']

[1, 0, 0, 0, 0]

['This is the 2nd time we have tried 2 contact u. U have won the £750 Pound prize. 2 claim is easy, call 087187272008 NOW1! Only 10p per minute. BT-national-rate.',
 'Will ü b going to esplanade fr home?',
 'Pity, * was in mood for that. So...any other suggestions?',
 "The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free",
 'Rofl. Its true to its name']

### Задание 4

Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.

In [58]:
# CountVectorizer for sms_text
count_vect = CountVectorizer()
X_counts = count_vect.fit_transform(sms_text) # fot_trasform return the matrix X of words counts
X_counts.shape

(5574, 8713)

### Задание 5

Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

In [63]:
# Building pipeline
pipeline = Pipeline([("vectorizer", CountVectorizer()), ("classifier", LogisticRegression())]
        )

In [81]:
score = cross_val_score(pipeline, sms_text, sms_label_bi, scoring='f1', cv=10).mean()
print('Cross_val_score f1: {0:.4f}'.format(score))

Cross_val_score f1: 0.9311


In [85]:
with open('answer_1.txt','w') as file_out:
    file_out.write(str(round(score,1)))

3

### Задание 6

А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

 "FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"

"FreeMsg: Txt: claim your reward of 3 hours talk time"

"Have you visited the last lecture on physics?"

"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

"Only 99$"

Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

In [74]:
sms_test = ["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
            "FreeMsg: Txt: claim your reward of 3 hours talk time",
            "Have you visited the last lecture on physics?",
            "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
            "Only 99$"]

In [75]:
# Fitting clf in the full dataset
pipeline.fit(sms_text, sms_label_bi)

Pipeline(steps=[('vectorizer', CountVectorizer()),
                ('classifier', LogisticRegression())])

In [84]:
# predicting labels for sms_test
sms_test_predictions = pipeline.predict(sms_test)
sms_test_predictions

array([1, 1, 0, 0, 0])

In [86]:
with open('answer_2.txt','w') as file_out:
    file_out.write(' '.join([str(x) for x in sms_test_predictions]) )

9

### Задание 7

Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [96]:
# testing log_reg the model in different ngram ranges
ngram_range = [(2,2), (3,3), (1,3)]

scores = []
for ngram in ngram_range:
    pipeline = Pipeline([("vectorizer", CountVectorizer(ngram_range=ngram)), ("classifier", LogisticRegression())])
    score = cross_val_score(pipeline, sms_text, sms_label_bi, scoring='f1', cv=10).mean()
    scores.append(score)
    print('Nram range: ', ngram)
    print('Cross_val_score f1: {0:.4f}\n'.format(score))

Nram range:  (2, 2)
Cross_val_score f1: 0.8168

Nram range:  (3, 3)
Cross_val_score f1: 0.7250

Nram range:  (1, 3)
Cross_val_score f1: 0.9223



In [94]:
with open('answer_3.txt','w') as file_out:
    file_out.write(' '.join([str(round(x,2)) for x in scores]))

14

### Задание 8

Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

 По какой-то причине  обучение наивного байесовского классификатора через Pipeline происходит с ошибкой. Чтобы получить правильный ответ, отдельно  посчитайте частоты слов и обучите классификатор.  

In [97]:
# testing log_reg the model in different ngram ranges
ngram_range = [(2,2), (3,3), (1,3)]

scores = []
for ngram in ngram_range:
    pipeline = Pipeline([("vectorizer", CountVectorizer(ngram_range=ngram)), ("classifier", MultinomialNB())])
    score = cross_val_score(pipeline, sms_text, sms_label_bi, scoring='f1', cv=10).mean()
    scores.append(score)
    print('Nram range: ', ngram)
    print('Cross_val_score f1: {0:.4f}\n'.format(score))

Nram range:  (2, 2)
Cross_val_score f1: 0.9344

Nram range:  (3, 3)
Cross_val_score f1: 0.8712

Nram range:  (1, 3)
Cross_val_score f1: 0.9472



In [100]:
# Because of the mistake in pipeline calculations we have to use the other approach
ngram_range = [(2,2), (3,3), (1,3)]

scores = []
for ngram in ngram_range:
    X_counts = CountVectorizer(ngram_range=ngram).fit_transform(sms_text)
    score = cross_val_score(MultinomialNB(), X_counts, sms_label_bi, scoring='f1', cv=10).mean()
    scores.append(score)
    print('Nram range: ', ngram)
    print('Cross_val_score f1: {0:.4f}\n'.format(score))

Nram range:  (2, 2)
Cross_val_score f1: 0.6451

Nram range:  (3, 3)
Cross_val_score f1: 0.3786

Nram range:  (1, 3)
Cross_val_score f1: 0.8878



In [102]:
with open('answer_4.txt','w') as file_out:
    file_out.write(' '.join([str(round(x,2)) for x in scores]))

14

### Задание 9

Попробуйте использовать в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [103]:
# Building pipeline with TFidf
pipeline = Pipeline([("vectorizer", TfidfVectorizer()), 
                     ("classifier", LogisticRegression())]
        )

In [104]:
score = cross_val_score(pipeline, sms_text, sms_label_bi, scoring='f1', cv=10).mean()
print('Cross_val_score f1: {0:.4f}'.format(score))

Cross_val_score f1: 0.8784


In [105]:
with open('answer_5.txt','w') as file_out:
    file_out.write('-1')

2